In [6]:
# x_1 = 7, x_2 = 6, s_3 = 0, s_4 = 0
obj = [5, 3]
consts = [[[3, 2],2400],[[0, 1],800], [[2, 0],1200]]
#basis [s1, s2]

In [7]:
def simplexMethod(objFunction: list, constraints: list, operations: list[int], isMax : bool = True) -> list:
    
    """
    Solves the linear programming problem using the Simplex method.

    Parameters:
    @param obj_function (list): The coefficients of the objective function.
    @param constraints (list): A list of constraints, where each constraint is a list containing the coefficients and the right-hand side value.

    Returns:
    list: The solution to the linear programming problem.
    """

    originalNum = len(constraints)
    internalCount = 0
    originalObj = objFunction.copy()

    # * Add slack variables
    for i in range(originalNum):
        objFunction.append(0)
        for j in range(originalNum):
            if internalCount%(len(constraints)+1) == 0:
                if operations[i] == 1: # <=
                    constraints[j][0].append(1)
                if operations[i] == 2: # >=
                    constraints[j][0].append(-1)
                else: # =
                    constraints[j][0].append(0)
            else:
                constraints[i][0].append(0)
            internalCount += 1
    print
    # * add artificial variables
    artificialVariableIndex = []
    for i in range(len(operations)):
        if operations[i] != 1:
            artificialVariableIndex.append(i)

    print(artificialVariableIndex)
    counter = 0
    checker = False
    for i in range(len(constraints)):
        if i in artificialVariableIndex:
            checker = True
        for j in range(len(artificialVariableIndex)):
            if i in artificialVariableIndex and j == counter and checker:
                constraints[i][0].append(1)
                counter += 1
                checker = False
            else :
                constraints[i][0].append(0)
    for i in range(len(artificialVariableIndex)):
        objFunction.append(0)
    
    for consts in constraints:
        print(consts)

    result = []
    resultIndex = [-1 for i in range(len(constraints))]
    basis = [0 for i in range(len(constraints))]
    z_j = [sum(basis[j]*constraints[j][0][i] for j in range(len(constraints))) for i in range(len(constraints[0][0]))]

    c_j_minus_z_j = [objFunction[i] - z_j[i] for i in range(len(z_j))]

    while max(c_j_minus_z_j) > 0:  
        maxIndex = c_j_minus_z_j.index(max(c_j_minus_z_j))

        ratios = []
        for i in range(len(constraints)):
            if constraints[i][0][maxIndex] > 0:
                ratios.append((constraints[i][1]/constraints[i][0][maxIndex], i))
        
        if not ratios:
            return "Unbounded solution"
            
        minRatio, minIndex = min(ratios)

        basis[minIndex] = objFunction[maxIndex]
        resultIndex[minIndex] = maxIndex


        pivot = constraints[minIndex][0][maxIndex]

        old_row = constraints[minIndex].copy()

        constraints[minIndex][0] = [x/pivot for x in constraints[minIndex][0]]
        constraints[minIndex][1] = constraints[minIndex][1]/pivot

        for i in range(len(constraints)):
            if i != minIndex:
                multiplier = constraints[i][0][maxIndex]
                constraints[i][0] = [constraints[i][0][j] - multiplier*constraints[minIndex][0][j] 
                                   for j in range(len(constraints[0][0]))]
                constraints[i][1] = constraints[i][1] - multiplier*constraints[minIndex][1]
        
        z_j = [sum(basis[j]*constraints[j][0][i] for j in range(len(basis))) for i in range(len(constraints[0][0]))]
        c_j_minus_z_j = [objFunction[i] - z_j[i] for i in range(len(z_j))]

    for i in range(len(originalObj)):
        if i in resultIndex:
            result.append(constraints[resultIndex.index(i)][1])
        else:
            result.append(0)
    objFunction = []
    constraints = []
    return result



In [8]:
profit = [3, 5, 4]
constraintProfit = [[[2, 3, 0],8],[[3, 2, 2], 15],[[0, 4,5], 10]]


In [9]:
import re

def parseInputConstrict(input_string: str, objVariables: list[str]) -> tuple:
    """
    Parses a linear equation input string and extracts the coefficients and the result.\n
    Args:\n
        input_string (str): The input string representing a linear equation. \n
                            The equation should be in the form of 'ax + by + cz = d' \n
                            where a, b, c, and d are numbers, and x, y, z are variables.\n
    Returns:\n
        tuple: A tuple containing two elements:\n
            - A list of coefficients (float) for each variable in the equation.\n
            - A float representing the result of the equation.
    """

    

    input_string = input_string.replace(" ", "")
    terms = re.findall(r'([+-]?\d*\.?\d*)([a-zA-Z]+)', input_string)
    coefficients = []
    for var in objVariables:
        coefficient = 0
        for term in terms:
            if term[1] == var:
                coefficient = term[0]
                if coefficient in ['+', '-'] or coefficient == '':
                    coefficient += '1'
                coefficient = float(coefficient)
                break
        coefficients.append(coefficient)
    result = input_string.split('=')[-1]
    return [coefficients, float(result)]
equation = "2a + 3b - 4c = 10"
coefficients = parseInputConstrict(equation, ['a', 'e', 'b', 'c'])
print(coefficients)

[[2.0, 0, 3.0, -4.0], 10.0]


In [10]:
def parseInputObj(input_string: str) -> tuple:
    """
    Parses a linear equation input string and extracts the coefficients and the result.\n
    Args:\n
        input_string (str): The input string representing a linear equation. \n
                            The equation should be in the form of 'ax + by + cz = d' \n
                            where a, b, c, and d are numbers, and x, y, z are variables.\n
    Returns:\n
        tuple: A tuple containing two elements:\n
            - A list of coefficients (float) for each variable in the equation.\n
            - A float representing the result of the equation.
    """
    input_string = input_string.replace(" ", "")
    terms = re.findall(r'([+-]?\d*\.?\d*)([a-zA-Z]+)', input_string)
    coefficients = []
    variables = []
    for term in terms:
        coefficient = term[0]
        variable = term[1]
        if coefficient in ['+', '-'] or coefficient == '':
            coefficient += '1'
        coefficients.append(float(coefficient))
        variables.append(variable)
    return coefficients, variables
equation = "20a + 3b - 4c"
coefficients = parseInputObj(equation)
print(coefficients)

([20.0, 3.0, -4.0], ['a', 'b', 'c'])


In [11]:
def simplexFunction(obj:list, constrict:list[list,int]) -> list:
    """
    Applies the simplex method to solve a linear programming problem.\n
    Parameters:\n
    obj (list): A list representing the objective function coefficients.\n
    constrict (list[list, int]): A list of constraints, where each constraint is represented 
                                 as a list containing the coefficients and the right-hand side value.\n
    Returns:\n
    list: The solution to the linear programming problem, typically including the optimal values 
          of the decision variables and the optimal value of the objective function.
    """

    obj = parseInputObj(obj)
    return simplexMethod(obj[0], [parseInputConstrict(constraint, obj[1]) for constraint in constrict])

obj = "3a + 5b + 4c + 3d"
constrict = ["2a + 3b + 4d = 10", "3a + 2b + 2c = 15", "a + 4b + 5c = 10"]
print(simplexFunction(obj, constrict))

TypeError: simplexMethod() missing 1 required positional argument: 'operations'

In [18]:
def northWestCornerRule(demand, supply, costValueMatrix):
    """ 
    
    1 1 1
    2 2 1
    3 1 1
    
    """

    #DEMAND is the y axis
    #SUPPLY is the x axis

    if sum(demand) != sum(supply):
        return "Invalid input"
    values = []
    keyPairValues = []
    Done = True
    i = 0 
    j = 0
    while Done:
        if supply[i] > demand[j]:
            values.append([demand[j], costValueMatrix[i][j]])
            keyPairValues.append([i, j])
            supply[i] -= demand[j]
            demand[j] = 0
            j += 1
        elif supply[i] < demand[j]:
            values.append([supply[i], costValueMatrix[i][j]])
            keyPairValues.append([i, j])
            demand[j] -= supply[i]
            supply[i] = 0
            i += 1
        elif supply[i] == demand[j]:
            values.append([supply[i], costValueMatrix[i][j]])
            keyPairValues.append([i, j])
            supply[i] = 0
            demand[j] = 0
            Done = False
    results = [value[0]*value[1] for value in values]
    return sum(results), keyPairValues

demand = [10, 20, 30]
supply = [20, 30, 10]
costValueMatrix = [[1, 2, 3], [1, 2, 1], [3, 1, 1]]
values, keyPairValues = northWestCornerRule(demand, supply, costValueMatrix)
print(values, keyPairValues)

80 [[0, 0], [0, 1], [1, 1], [1, 2], [2, 2]]


In [1]:
def leastCostRule(demand, supply, costValueMatrix):
    if sum(demand) != sum(supply):
        return "Invalid input"
    adjustedCostValueMatrix = costValueMatrix.copy()
    values = []
    keyPairValues = []
    #m is the demand or y axis
    #n is the supply or x axis
    m = 0
    n = 0
    Done = False
    while not Done:
        minCost = float('inf')
        for row in adjustedCostValueMatrix:
            if min(row) < minCost:
                minCost = min(row)
                m = adjustedCostValueMatrix.index(row)
                n = row.index(min(row))
        print(minCost, m, n)
        if demand[m] > supply[n]:
            values.append([supply[n], minCost])
            keyPairValues.append([m, n])
            demand[m] -= supply[n]
            for i in range(len(demand)):
                adjustedCostValueMatrix[i][n] = float('inf')
            supply[n] = 0

        elif demand[m] < supply[n]:
            values.append([demand[m], minCost])
            keyPairValues.append([m, n])
            supply[n] -= demand[m]
            for i in range(len(supply)):
                adjustedCostValueMatrix[m][i] = float('inf')
            demand[m] = 0
        elif demand[m] == supply[n]:
            values.append([demand[m], minCost])
            keyPairValues.append([m, n])
            demand[m] = 0
            supply[n] = 0
        if sum(demand) == 0 and sum(supply) == 0:
            Done = True
        print(adjustedCostValueMatrix)
    results = [value[0]*value[1] for value in values]
    return sum(results), keyPairValues


demand = [10, 20, 15]
supply = [12, 18, 15]
costValueMatrix = [[2,1,3],[4,5,7],[1,8,6]]
values, keyPairValues = leastCostRule(demand, supply, costValueMatrix)
print(values, keyPairValues)

1 0 1
[[inf, inf, inf], [4, 5, 7], [1, 8, 6]]
1 2 0
[[inf, inf, inf], [inf, 5, 7], [inf, 8, 6]]
5 1 1
[[inf, inf, inf], [inf, inf, 7], [inf, inf, 6]]
6 2 2
[[inf, inf, inf], [inf, inf, 7], [inf, inf, inf]]
7 1 2
[[inf, inf, inf], [inf, inf, 7], [inf, inf, inf]]
164 [[0, 1], [2, 0], [1, 1], [2, 2], [1, 2]]


In [6]:
def vogelApproximation(costValueMatrix, demand, supply):
    if len(costValueMatrix) != len(demand):
        y = supply.copy()
        x = demand.copy()
    else:
        y = demand.copy()
        x = supply.copy()
    penalty_y = []
    penalty_x = []
    values = []
    keyPairValues = []
    x_index = 0
    y_index = 0
    Done = False
    for row in costValueMatrix:
        penalty_y.append(sorted(row)[1] - sorted(row)[0])
    for i in range(len(costValueMatrix[0])):
        column = [row[i] for row in costValueMatrix]
        penalty_x.append(sorted(column)[1] - sorted(column)[0])
    while not Done:
        if max(penalty_y) > max(penalty_x):
            y_index = penalty_y.index(max(penalty_y))
            minCost = min(costValueMatrix[y_index])
            x_index = costValueMatrix[y_index].index(minCost)
            if x[x_index] > y[y_index]:  # choose y
                values.append([y[y_index], minCost])
                keyPairValues.append([y_index, x_index])
                x[x_index] -= y[y_index]
                for i in range(len(costValueMatrix[0])):
                    costValueMatrix[y_index][i] = float('inf')
                y[y_index] = 0
            elif x[x_index] < y[y_index]:  # choose x
                values.append([x[x_index], minCost])
                keyPairValues.append([y_index, x_index])
                y[y_index] -= x[x_index]
                for i in range(len(costValueMatrix)):
                    costValueMatrix[i][x_index] = float('inf')
                x[x_index] = 0
            elif x[x_index] == y[y_index]:  # choose x
                values.append([x[x_index], minCost])
                keyPairValues.append([y_index, x_index])
                y[y_index] -= x[x_index]
                for i in range(len(costValueMatrix)):
                    costValueMatrix[i][x_index] = float('inf')
                x[x_index] = 0
            penalty_y[y_index] = float('-inf')
        elif max(penalty_y) < max(penalty_x):
            x_index = penalty_x.index(max(penalty_x))
            minCost = float('inf')
            for row in costValueMatrix:
                if min(row) < minCost:
                    minCost = min(row)
                    y_index = costValueMatrix.index(row)
            if x[x_index] > y[y_index]:  # choose y
                values.append([y[y_index], minCost])
                keyPairValues.append([y_index, x_index])
                x[x_index] -= y[y_index]
                for i in range(len(costValueMatrix[0])):
                    costValueMatrix[y_index][i] = float('inf')
                y[y_index] = 0
            elif x[x_index] < y[y_index]:  # choose x
                values.append([x[x_index], minCost])
                keyPairValues.append([y_index, x_index])
                y[y_index] -= x[x_index]
                for i in range(len(costValueMatrix)):
                    costValueMatrix[i][x_index] = float('inf')
                x[x_index] = 0
            elif x[x_index] == y[y_index]:  # choose x
                values.append([x[x_index], minCost])
                keyPairValues.append([y_index, x_index])
                y[y_index] -= x[x_index]
                for i in range(len(costValueMatrix)):
                    costValueMatrix[i][x_index] = float('inf')
                x[x_index] = 0
            penalty_x[x_index] = float('-inf')
        elif max(penalty_y) == max(penalty_x):
            y_index = penalty_y.index(max(penalty_y))
            minCost = min(costValueMatrix[y_index])
            x_index = costValueMatrix[y_index].index(minCost)
            if x[x_index] > y[y_index]:  # choose y
                values.append([y[y_index], minCost])
                keyPairValues.append([y_index, x_index])
                x[x_index] -= y[y_index]
                for i in range(len(costValueMatrix[0])):
                    costValueMatrix[y_index][i] = float('inf')
                y[y_index] = 0
            elif x[x_index] < y[y_index]:  # choose x
                values.append([x[x_index], minCost])
                keyPairValues.append([y_index, x_index])
                y[y_index] -= x[x_index]
                for i in range(len(costValueMatrix)):
                    costValueMatrix[i][x_index] = float('inf')
                x[x_index] = 0
            elif x[x_index] == y[y_index]:  # choose x
                values.append([x[x_index], minCost])
                keyPairValues.append([y_index, x_index])
                y[y_index] -= x[x_index]
                for i in range(len(costValueMatrix)):
                    costValueMatrix[i][x_index] = float('inf')
                x[x_index] = 0
            penalty_y[y_index] = float('-inf')
        if sum(x) <= 0 and sum(y) <= 0:
            Done = True
        for row in costValueMatrix:
            if y[costValueMatrix.index(row)] > 0:
                penalty_y[costValueMatrix.index(row)] = sorted(row)[1] - sorted(row)[0]
        for i in range(len(costValueMatrix[0])):
            if x[i] > 0:
                column = [row[i] for row in costValueMatrix]
                penalty_x[i] = sorted(column)[1] - sorted(column)[0]
    results = [value[0] * value[1] for value in values]
    return sum(results), keyPairValues


demand = [5, 8, 7, 14]
supply = [7, 9, 18]
costValueMatrix = [[19, 30, 50, 10], [70, 30, 40, 60], [40, 8, 70, 20]]
values, keyPairValues = vogelApproximation(costValueMatrix, demand, supply)
print(values, keyPairValues)


734 [[2, 1], [0, 0], [2, 3], [0, 3], [1, 2], [1, 3]]
